In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.5 MB/s eta 0:00:00


In [3]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/Raiden-Makoto/Genshinfy-UwU

Cloning into 'Genshinfy-UwU'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 168 (delta 31), reused 125 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (168/168), 37.09 MiB | 18.29 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [5]:
%cd Genshinfy-UwU

/content/Genshinfy-UwU


In [6]:
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from datasets import Image as IMG
from typing import List, Union, Tuple
from transformers import CLIPProcessor, CLIPModel

import faiss
import glob

In [12]:
img_path = './GenshinCharacters'
img_files = glob.glob(img_path + "/*.[jJpP][pPnN][gG]")

In [14]:
img_files = sorted(img_files)
img_files[0]

'./GenshinCharacters/Albedo.png'

In [16]:
device = "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [17]:
def encode_images(
    images: Union[List[str], List[PIL.Image.Image]],
    batch_size: int
):
    def transform_func(x):
        if isinstance(x['image'], PIL.Image.Image):
            image = x['image']
        else:
            image = [IMG().decode_example(img) for img in x['image']]
        return processor(images=image, return_tensors='pt')

    dataset = Dataset.from_dict({'image' : images})
    dataset = dataset.cast_column('image', IMG(decode=False)) \
                if isinstance(images[0], str) \
                else dataset
    dataset.set_format('torch')
    dataset.set_transform(transform_func)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    image_embeddings = []
    progress = tqdm(total=len(images)//batch_size, position=0)

    with torch.no_grad():
        for batch in dataloader:
            batch = {k : v.to(device) for k,v in batch.items()}
            image_embeddings.extend(
                model.get_image_features(**batch).detach().cpu().numpy()
            )
            progress.update(1)
        progress.close()
    return np.stack(image_embeddings)

In [18]:
vector_embedding = np.array(encode_images(img_files, 7))

14it [00:20,  1.46s/it]


In [21]:
# Store Embeddings
with open('vector_embedding.pkl', 'wb') as f:
    pickle.dump(vector_embedding, f)

In [24]:
indexer = faiss.IndexFlatIP(vector_embedding.shape[1]) #inner product similarity
indexer.add(vector_embedding)

In [40]:
def encode_text(
    text: List[str],
    batch_size: int
):
    dev = "cpu"
    dataset = Dataset.from_dict({'text' : text})
    dataset = dataset.map(
        lambda x: processor(
            text = x['text'],
            padding = True,
            return_tensors = 'pt',
            max_length = 77,
            truncation = True,
        ),
        batched=True,
        batch_size=batch_size
    )
    dataset.set_format('torch')
    dataloader = DataLoader(dataset, batch_size=batch_size)
    text_embeddings = []
    progress = tqdm(total=len(text) // batch_size, position=0)

    with torch.no_grad():
        for batch in dataloader:
            for k, v in batch.items():
                print(k, v, type(v))  # Check data types
            batch = {k : v.to(dev) for k,v in batch.items()}
            text_embeddings.extend(
                model.get_text_features(**batch).detach().cpu().numpy()
            )
            progress.update(1)
        progress.close()
    return np.stack(text_embeddings)


In [41]:
search_text = input("Enter some text: ").strip()

Enter some text: cat


In [42]:
with torch.no_grad():
    search_results = encode_text([search_text], batch_size=7)
search_restuls = search_results / np.linalg.norm(search_results, ord=2, axis=-1, keepdims=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

text ['cat'] <class 'list'>
input_ids tensor([[49406,  2368, 49407]]) <class 'torch.Tensor'>
attention_mask tensor([[1, 1, 1]]) <class 'torch.Tensor'>


AttributeError: 'list' object has no attribute 'to'